In [ ]:
import tensorflow as tf
import numpy as np
import imutils
from imutils import paths
import cv2
import random
import pickle
import os
import sklearn
import matplotlib 

# Load a random image and it's label
imagePaths = list(paths.list_images("dataset"))
imagePath = random.choice(imagePaths)

trueLabel = imagePath.split(os.path.sep)[-2]
image = cv2.imread(imagePath)
orig = image.copy()

# pre-process the image for classification
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224,224))
image = image.astype("float") / 255.0
image = np.expand_dims(image, axis=0)

# Load model
model = tf.keras.models.load_model("covid19.model")

# Load labels
lb = pickle.load(open("labels.pickle", "rb"))

# Classify image
preds = model.predict(image)

# find the class label index with the largest probability
i = preds.argmax(axis=1)[0]
label = lb.classes_[i]

# build texts
realText = f"Real: {trueLabel}"
predText = "Pred: {} ({:.2f}%)".format(label, preds[0][i] * 100)

# print texts
print(realText)
print(predText)
print(imagePath)
# draw texts
output = imutils.resize(orig, width=400)
cv2.putText(output, realText, (10, 55),  cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (0, 255, 0), 2)
cv2.putText(output, predText, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (0, 255, 0), 2)

# show the output image
cv2.imshow("Output", output)
cv2.waitKey(0)



In [1]:
pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import anvil.server

In [2]:
anvil.server.connect("PR2H6U7FTAJTXOZHGZR2PSAD-OE35PIJXBCRB3GR5")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [3]:
import tensorflow as tf
import numpy as np
import imutils
from imutils import paths
import cv2
import random
import pickle
import os
import sklearn
import matplotlib 
import anvil.media

@anvil.server.callable
def classify_image(file):
    with anvil.media.TempFile(file) as filename:
        img = cv2.imread(filename)
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = img.astype("float") / 255.0
    img = np.expand_dims(img, axis=0)
    # Load model
    model = tf.keras.models.load_model("covid19.model")

    # Load labels
    lb = pickle.load(open("labels.pickle", "rb"))

    # Classify image
    preds = model.predict(img)

    # find the class label index with the largest probability
    i = preds.argmax(axis=1)[0]
    label = lb.classes_[i]

    # build texts
    realText = " "
    predText = "Prediction: {} ({:.2f}%)".format(label, preds[0][i] * 100)
    
    return(label, predText)

C:\Users\Ezgi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
